# Natural Language Processing and Text Analytics Final Exam [KAN-CDSCO1002U]

Afnan El-Segaier (137863)  
Une Aspelin (152294)  
Kristin Sundby (167303)  
Marlin Haavengen (167342)



---



This section outlines the code for the **BERT** model:
*   Preprocessing
*   Building the model using the training dataset
*   Tokenization
*   Testing the model using the test dataset
*   Cross Validation
*   Tuning



## BERT Model

### **Step 1: Preprocessing:**

In [ ]:
# Importing the necessary libraries:
#pip install transformers
#pip install torch
#pip install optuna
import pandas as pd
import numpy as np
import re
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold
import optuna
from transformers import BertConfig, BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

/Users/marlinhaavengen/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/marlinhaavengen/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Loading the training and test datasets:
df_train = pd.read_csv('isarcasm_train.csv')
df_test = pd.read_csv('isarcasm_test.csv')

In [ ]:
# Renaming the columns in both train and test to tweet and label for consistency:
df_train.columns = ['tweet', 'label']
df_test.columns = ['tweet', 'label']

**Quick Data Inspection:**

In [ ]:
df_train["label"].value_counts()

0    2601
1     867
Name: label, dtype: int64

In [ ]:
df_test["label"].value_counts()

0    1200
1     200
Name: label, dtype: int64

In [ ]:
# Check for missing values in the train set
df_train.isnull().sum()

tweet    1
label    0
dtype: int64

In [ ]:
# Check for missing values in the test set
df_test.isnull().sum()

tweet    0
label    0
dtype: int64

**Preparing The Data:**

In [ ]:
# Drop the missing value in the train set
df_train.dropna(inplace=True)

In [ ]:
# Combine the datasets to be used for cross-validation
df_full = pd.concat([df_train, df_test], ignore_index=True)

### **Step 2: Build The Model**

In [ ]:
# Defining test and train data:
X_train = df_train['tweet']
y_train = df_train['label']
X_test = df_test['tweet']
y_test = df_test['label']

In [ ]:
# Model Initialization:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Load pre-trained BERT tokenizer:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Printing Model And Tokenizer For Inspection:**

In [ ]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
print(tokenizer)

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


###**Step 3: Tokenize The Data And Convert To PyTorch Dataset**

**Inspection of Token Lengths:**  
To be used in the tokenizer function.

In [ ]:
# Find max token length
max_token_length_train = max(len(tokenizer.encode(tweet)) for tweet in df_train['tweet'])
max_token_length_test = max(len(tokenizer.encode(tweet)) for tweet in df_test['tweet'])
print("Maximum token length in training data:", max_token_length_train)
print("Maximum token length in test data:", max_token_length_test)


Maximum token length in training data: 116
Maximum token length in test data: 146


**Defining And Applying Tokenizer Function:**

In [ ]:
# Defining a function to tokenize input texts using a tokenizer:
# The function returns a dictionary containing the tokenized input sentences, attention mask and other info BERT requires:

def tokenize_data(texts, max_length=150): # Setting the maximum length of the tokenized sequences to 150, as that will ensure that all are included.
    return tokenizer(
        texts.tolist(),  # Converting input to a list if they are not already.
        padding="max_length",  # Padding all sequences to the maximum length.
        truncation=True,  # Truncate sequences to the maximum length if they exceed it.
        max_length=max_length,  # Specifying the maximum length for the tokenized sequences.
        return_tensors="pt"  # Return the tokenized sequences as PyTorch tensors.
    )

In [ ]:
# Applying the tokenizer function to the training and testing data:
train_encodings = tokenize_data(X_train)
test_encodings = tokenize_data(X_test)

**Converting Token IDs For Inspection:**

In [ ]:
#  Printing the token IDs for the first tweet:
print("Token IDs for the first tweet:", train_encodings['input_ids'][0])

Token IDs for the first tweet: tensor([  101,  1996,  2069,  2518,  1045,  2288,  2013,  2267,  2003,  1037,
        24689,  7959,  3170, 13449,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     

In [ ]:
# Converting token IDs back to tokens and printing the tokens for the first tweet:
print("Tokens for the first tweet:", tokenizer.convert_ids_to_tokens(train_encodings['input_ids'][0]))

Tokens for the first tweet: ['[CLS]', 'the', 'only', 'thing', 'i', 'got', 'from', 'college', 'is', 'a', 'caf', '##fe', '##ine', 'addiction', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PA

In [ ]:
# Move model to GPU if available
#device = torch.device(
#	'cuda') if torch.cuda.is_available() else torch.device('cpu')
#model = model.to(device)

**Defining And Applying Function Converting To PyTorch:**

In [ ]:
# Defining the 'SarcasmDataset' class which converts tokenized data and labels into a format that can be used with PyTorch's data loaders.
class SarcasmDataset(Dataset):
    def __init__(self, encodings, labels):
      # Initializing the dataset with encodings and labels:
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # To retrieve an item by the index.
        item = {key: self.encodings[key][idx] for key in self.encodings}  # Get the tokenized input at the specified index.
        item['labels'] = self.labels[idx]  # Add the corresponding label to the item.
        return item  # Return the dictionary containing the tokenized input and label.

    def __len__(self):
        # Return the length of the dataset (the number of samples).
        return len(self.labels)  # The length is based on the number of labels.

In [ ]:
# Applying the function:
train_dataset = SarcasmDataset(train_encodings, y_train.values)
test_dataset = SarcasmDataset(test_encodings, y_test.values)

## **Step 4: Train The Model On The Data**

In [ ]:
#pip install transformers[torch] --upgrade

In [ ]:
#pip install accelerate -U

In [ ]:
# Defining the training arguments:
training_args = TrainingArguments(
    output_dir='./results',          # Where to save the model and logs
    num_train_epochs=3,              # Number of training epochs
    per_device_train_batch_size=8,   # Batch size for each device during training
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,                # Log training info every 10 steps
    evaluation_strategy="no"         # Evaluate only at the end
)

**Initializing the Trainer class form the Hugging Face library, to set op the training process:**

In [ ]:
# Specifying the model, arguments and dataset:
trainer = Trainer(
    model=model,                  # The pre-trained BERT-model to be fine-tuned
    args=training_args,           # The training arguments defined above
    train_dataset=train_dataset   # Specifying that the model is to be trained on the train dataset
)

**Starting the training process using the Trainer Class:**  
Handles the entire training loop, including the forward and backward pass, and optimization steps over the training dataset.  
It applies the arguments specified above.

In [ ]:
trainer.train() # Initiates the training loop and describes what the method handles during the training process.

  0%|          | 0/1302 [00:00<?, ?it/s]

  1%|          | 10/1302 [00:11<22:25,  1.04s/it]

{'loss': 0.6447, 'grad_norm': 5.92990255355835, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.02}


  2%|▏         | 20/1302 [00:19<17:36,  1.21it/s]

{'loss': 0.5957, 'grad_norm': 5.627193927764893, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.05}


...

100%|█████████▉| 1300/1302 [24:31<00:01,  1.25it/s]

{'loss': 0.1635, 'grad_norm': 0.7562901377677917, 'learning_rate': 1.2468827930174563e-07, 'epoch': 3.0}


100%|██████████| 1302/1302 [24:32<00:00,  1.13s/it]

{'train_runtime': 1472.646, 'train_samples_per_second': 7.063, 'train_steps_per_second': 0.884, 'train_loss': 0.4446687590353729, 'epoch': 3.0}


TrainOutput(global_step=1302, training_loss=0.4446687590353729, metrics={'train_runtime': 1472.646, 'train_samples_per_second': 7.063, 'train_steps_per_second': 0.884, 'total_flos': 801743580117000.0, 'train_loss': 0.4446687590353729, 'epoch': 3.0})

**Predict on Train And Test Data:**

In [ ]:
# Predicting on the train set:
predictions_train = trainer.predict(train_dataset)

# Extracting the predicted labels:
pred_labels_train = np.argmax(predictions_train.predictions, axis=-1)

# Predicting on the test set:
predictions_test = trainer.predict(test_dataset)

# Extracting the predicted labels:
pred_labels_test = np.argmax(predictions_test.predictions, axis=-1)

100%|██████████| 175/175 [00:35<00:00,  4.95it/s]


**Printing Confusion Matrices And Classification Reports For Evaluation:**

In [ ]:
print("Classification Report Train set: \n", classification_report(y_train, pred_labels_train))
print("Confusion Matrix Train set: \n", confusion_matrix(y_train, pred_labels_train))

Classification Report Train set: 
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      2600
           1       0.99      0.91      0.95       867

    accuracy                           0.98      3467
   macro avg       0.98      0.95      0.97      3467
weighted avg       0.98      0.98      0.97      3467

Confusion Matrix Train set: 
 [[2592    8]
 [  78  789]]


In [ ]:
print("Classification Report Test set: \n", classification_report(y_test, pred_labels_test))
print("Confusion Matrix Test set: \n", confusion_matrix(y_test, pred_labels_test))

Classification Report Test set: 
               precision    recall  f1-score   support

           0       0.90      0.84      0.87      1200
           1       0.30      0.42      0.35       200

    accuracy                           0.78      1400
   macro avg       0.60      0.63      0.61      1400
weighted avg       0.81      0.78      0.79      1400

Confusion Matrix Test set: 
 [[1005  195]
 [ 115   85]]


The model is performing very well on the training data but much less so on the test data, suggesting overfitting. Therefore we will performe Cross Validation and Hyperparameter Tuning.

## **Step 5: Cross Validation**

**Utilizing K-Fold Cross-Validation:**

In [ ]:
# Initializeing the KFold cross-validator with 5 splits, shuffling the data before splitting, and setting a random seed for reproducibility:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

results = [] # List to store evaluation results for each fold
confusion_matrices_val = [] # List to store confusion matrices for validation data for each fold

# Loop over each fold generated by KFold
for fold, (train_index, val_index) in enumerate(kf.split(df_full)):
    # Split data into training and validation sets for the current fold
    X_train, X_val = df_full.iloc[train_index]['tweet'], df_full.iloc[val_index]['tweet']
    y_train, y_val = df_full.iloc[train_index]['label'], df_full.iloc[val_index]['label']

    # Tokenizing the training and validation data:
    train_encodings = tokenize_data(X_train)
    val_encodings = tokenize_data(X_val)

    # Preparing datasets for the Trainer:
    train_dataset = SarcasmDataset(train_encodings, y_train.values)
    val_dataset = SarcasmDataset(val_encodings, y_val.values)

    # Reinitialize the BERT model for each fold to ensure independence:
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

    # Defining training arguments including output directories, number of epochs, batch size, evaluation strategy, logging settings, and learning rate:
    training_args = TrainingArguments(
        output_dir=f'./results_fold_{fold}', # Output directory for model checkpoints and other files
        num_train_epochs=3, # Number of training epochs = 3.
        per_device_train_batch_size=8, # Batch size for training.
        evaluation_strategy='epoch',  # Evaluate at the end of each epoch.
        logging_dir=f'./logs_fold_{fold}', # Directory for storing logs
        logging_steps=50, # Log every 50 steps
        learning_rate=5e-5, # Learning rate for the optimizer
    )

    # Initialize the Trainer with the model, training arguments, datasets, and optionally metrics computation:
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=None  # (Optionally define a compute_metrics function for evaluation metrics).
    )

    # Train the model on the training dataset:
    trainer.train()
    # Evaluating the model on the validation dataset:
    eval_result = trainer.evaluate()

    # Predicting on the validation dataset to obtain raw model predictions:
    predictions = trainer.predict(val_dataset)
    preds = np.argmax(predictions.predictions, axis=-1)

    # Computing the confusion matrix for the validation data predictions:
    cm_val = confusion_matrix(y_val, preds)
    report_val = classification_report(y_val, preds)

    # Storing the evaluation result and confusion matrix for this fold:
    results.append(eval_result)
    # Generating a detailed classification report for the validation data predictions:
    confusion_matrices_val.append(cm_val)

    print(f"Confusion Matrix for validation data for fold {fold+1}:\n{cm_val}\n")
    print(f"Classification Report for validation data for fold {fold+1}:\n{report_val}\n")

# After completing all folds, print the results for each fold:
for fold, result in enumerate(results):
    print(f"Fold {fold+1} results: {result}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  3%|▎         | 50/1461 [00:39<18:40,  1.26it/s]

{'loss': 0.549, 'grad_norm': 2.5183486938476562, 'learning_rate': 4.8288843258042436e-05, 'epoch': 0.1}


  7%|▋         | 100/1461 [01:21<19:02,  1.19it/s]

{'loss': 0.5109, 'grad_norm': 9.904523849487305, 'learning_rate': 4.657768651608487e-05, 'epoch': 0.21}


 10%|█         | 150/1461 [04:48<16:55,  1.29it/s]   

...

{'loss': 0.2934, 'grad_norm': 14.501498222351074, 'learning_rate': 3.7645448323066393e-07, 'epoch': 2.98}


                                                   
100%|██████████| 1461/1461 [26:24<00:00,  1.08s/it]


{'eval_loss': 0.6996310949325562, 'eval_runtime': 24.7558, 'eval_samples_per_second': 39.304, 'eval_steps_per_second': 4.928, 'epoch': 3.0}
{'train_runtime': 1584.0737, 'train_samples_per_second': 7.375, 'train_steps_per_second': 0.922, 'train_loss': 0.44801764305448954, 'epoch': 3.0}


100%|██████████| 122/122 [00:25<00:00,  4.78it/s]


Confusion Matrix for validation data for fold 5:
[[700  79]
 [115  79]]

Classification Report for validation data for fold 5:
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       779
           1       0.50      0.41      0.45       194

    accuracy                           0.80       973
   macro avg       0.68      0.65      0.66       973
weighted avg       0.79      0.80      0.79       973


Fold 1 results: {'eval_loss': 0.554473876953125, 'eval_runtime': 24.4562, 'eval_samples_per_second': 39.826, 'eval_steps_per_second': 4.989, 'epoch': 3.0}
Fold 2 results: {'eval_loss': 0.5243842005729675, 'eval_runtime': 25.9365, 'eval_samples_per_second': 37.553, 'eval_steps_per_second': 4.704, 'epoch': 3.0}
Fold 3 results: {'eval_loss': 0.47629255056381226, 'eval_runtime': 24.2951, 'eval_samples_per_second': 40.049, 'eval_steps_per_second': 5.022, 'epoch': 3.0}
Fold 4 results: {'eval_loss': 1.0272753238677979, 'eval_runtime': 23.1038, 'e

## **Step 6: Tuning**

In [ ]:
# Split the original training data into a smaller training subset (50% of original) and a validation subset:
train_subset, _ = train_test_split(train_dataset, test_size=0.5, random_state=42)
train_subset, valid_subset = train_test_split(train_subset, test_size=0.2, random_state=42)  # Further split into 80/20

**Optuna Hyperparameter Optimization:**

In [ ]:
# Defining the objective function for Optuna hyperparameter optimization:
def objective(trial):
    # Suggesting hyperparameters for the trial:
    num_train_epochs = trial.suggest_int('num_train_epochs', 1, 2) # Suggesting 1, 2 training epochs.
    per_device_train_batch_size = trial.suggest_categorical('per_device_train_batch_size', [8, 16]) # Batch size suggestion.
    learning_rate = trial.suggest_float('learning_rate', 2e-5, 5e-5) # Learning rate suggestion.
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-2) # Weight Decay suggestion.
    hidden_dropout_prob = trial.suggest_float('hidden_dropout_prob', 0.1, 0.3) # Suggesting hidden layer dropout probability.
    attention_probs_dropout_prob = trial.suggest_float('attention_probs_dropout_prob', 0.1, 0.3) # Suggesting attention dropout probability.
    warmup_steps = trial.suggest_int('warmup_steps', 100, 300) # Suggesting number of warmup steps.

    # Configuring the model with the suggested dropout rates:
    config = BertConfig.from_pretrained('bert-base-uncased',
                                        hidden_dropout_prob=hidden_dropout_prob,
                                        attention_probs_dropout_prob=attention_probs_dropout_prob)
    # Initializing the BERT for sequence classification:
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

    # Defining training arguments including output directory, number of epochs, batch size, learning rate, etc.:
    training_args = TrainingArguments(
        output_dir='./results', # Directory for saving.
        num_train_epochs=num_train_epochs, # Training epoch
        per_device_train_batch_size=per_device_train_batch_size, # Batch size per device.
        learning_rate=learning_rate, # Learning Rate.
        weight_decay=weight_decay, # Weigth decay for regularization.
        warmup_steps=warmup_steps, # Number of warmup steps.
        logging_dir='./logs', # Logging directory.
        logging_steps=10, # Log every 10 steps.
        evaluation_strategy='steps', # Model evaluation every few steps.
        eval_steps=50, # Evaluation every 50 step.
        save_total_limit=1, # Total number of checkpoints limit.
        load_best_model_at_end=True, # Loading the best model at the end of the training.
    )

    # Initializing the Trainer with the model, training arguments, and datasets:
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_subset, # The training subset.
        eval_dataset=valid_subset, # The validation subset.
    )

    # Train the model
    trainer.train()

    # Predicting on the validation set to get 'raw' predictions:
    predictions_valid = trainer.predict(valid_subset)
    # Get the predicted labels by taking the argmax of the prediction scores:
    pred_labels_valid = np.argmax(predictions_valid.predictions, axis=-1)

    # Calculate the evaluation metric, here using the error rate on the validation set:
    eval_loss = np.mean(predictions_valid.label_ids != pred_labels_valid)

    # Print the trial number and parameters
    print(f'Trial {trial.number} finished with value: {eval_loss} and parameters: {trial.params}')@

    # Return the evaluation loss as the objective to minimize:
    return eval_loss

In [ ]:
# Creating an Optuna study object with the objective to minimize the evaluation loss:
study = optuna.create_study(direction='minimize')

# Optimizing the objective function using the study object, specifying the number of trials to run:
study.optimize(objective, n_trials=5) # 5 trials.

# Print the best hyperparameters found during this process:
print("Best hyperparameters: ", study.best_params)

[I 2024-05-25 23:00:16,800] A new study created in memory with name: no-name-ea1652e0-9a9c-49b7-9978-fd36968625f7
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 11%|█▏        | 10/87 [03:02<22:46, 17.74s/it]

{'loss': 0.7212, 'grad_norm': 4.859196662902832, 'learning_rate': 3.5103003737191408e-06, 'epoch': 0.11}


 23%|██▎       | 20/87 [06:28<20:35, 18.43s/it]

{'loss': 0.6736, 'grad_norm': 6.755258083343506, 'learning_rate': 7.0206007474382816e-06, 'epoch': 0.23}


 34%|███▍      | 30/87 [09:22<15:29, 16.31s/it]

...

{'loss': 0.5834, 'grad_norm': 5.961061477661133, 'learning_rate': 2.596168026589056e-05, 'epoch': 0.98}


100%|██████████| 174/174 [04:33<00:00,  1.57s/it]


{'train_runtime': 273.1753, 'train_samples_per_second': 5.074, 'train_steps_per_second': 0.637, 'train_loss': 0.5873013912946329, 'epoch': 1.0}


100%|██████████| 44/44 [00:13<00:00,  3.16it/s]
[I 2024-05-26 00:50:57,824] Trial 4 finished with value: 0.2478386167146974 and parameters: {'num_train_epochs': 1, 'per_device_train_batch_size': 8, 'learning_rate': 3.894252039883584e-05, 'weight_decay': 0.0063986528296316484, 'hidden_dropout_prob': 0.10500577565336684, 'attention_probs_dropout_prob': 0.19239252680070668, 'warmup_steps': 255}. Best is trial 1 with value: 0.2420749279538905.


Trial 4 finished with value: 0.2478386167146974 and parameters: {'num_train_epochs': 1, 'per_device_train_batch_size': 8, 'learning_rate': 3.894252039883584e-05, 'weight_decay': 0.0063986528296316484, 'hidden_dropout_prob': 0.10500577565336684, 'attention_probs_dropout_prob': 0.19239252680070668, 'warmup_steps': 255}
Best hyperparameters:  {'num_train_epochs': 2, 'per_device_train_batch_size': 8, 'learning_rate': 3.248973807606919e-05, 'weight_decay': 0.006033468033489174, 'hidden_dropout_prob': 0.15698566483474938, 'attention_probs_dropout_prob': 0.19330502088918516, 'warmup_steps': 188}


In [ ]:
# After obtaining the best hyperparameters, train the model with the best parameters
best_params = study.best_params
config = BertConfig.from_pretrained('bert-base-uncased',
                                    hidden_dropout_prob=best_params['hidden_dropout_prob'],
                                    attention_probs_dropout_prob=best_params['attention_probs_dropout_prob'])
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Proceeding with the best hyperparameters found by Optuna:**

In [ ]:
# Define training arguments using the best hyperparameters found by Optuna
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=best_params['num_train_epochs'],  # Best number of training epochs from Optuna
    per_device_train_batch_size=best_params['per_device_train_batch_size'],  # Best batch size per device from Optuna
    learning_rate=best_params['learning_rate'],  # Best learning rate from Optuna
    weight_decay=best_params['weight_decay'],  # Best weight decay from Optuna
    logging_dir='./logs',  # Directory for logging the training information.
    logging_steps=50,  # Log every 50 steps.
    evaluation_strategy='no'  # No evaluation during training.
)


In [ ]:
# Initialize the Trainer again, this time with the best hyperparameters:
trainer = Trainer(
    model=model,
    args=training_args,  # Training arguments including best hyperparameters found by Optuna
    train_dataset=train_dataset
)


In [ ]:
# Training the final model with the best hyperparameters:
trainer.train()

  6%|▌         | 50/868 [00:42<10:53,  1.25it/s]

{'loss': 0.6148, 'grad_norm': 4.780924320220947, 'learning_rate': 3.0618209385051375e-05, 'epoch': 0.12}


 12%|█▏        | 100/868 [01:23<10:30,  1.22it/s]

{'loss': 0.5349, 'grad_norm': 2.5848705768585205, 'learning_rate': 2.8746680694033568e-05, 'epoch': 0.23}


 17%|█▋        | 150/868 [02:03<09:28,  1.26it/s]

...

 98%|█████████▊| 850/868 [11:47<00:14,  1.28it/s]

{'loss': 0.4922, 'grad_norm': 6.726139068603516, 'learning_rate': 6.737503287664117e-07, 'epoch': 1.96}


100%|██████████| 868/868 [27:32<00:00,  1.90s/it] 

{'train_runtime': 1652.8534, 'train_samples_per_second': 4.195, 'train_steps_per_second': 0.525, 'train_loss': 0.5370206420872069, 'epoch': 2.0}


TrainOutput(global_step=868, training_loss=0.5370206420872069, metrics={'train_runtime': 1652.8534, 'train_samples_per_second': 4.195, 'train_steps_per_second': 0.525, 'total_flos': 534495720078000.0, 'train_loss': 0.5370206420872069, 'epoch': 2.0})

In [ ]:
# Predicting on the train set and extracting the predicted labels:
predictions_train = trainer.predict(train_dataset)
pred_labels_train = np.argmax(predictions_train.predictions, axis=-1)

# Predicting on the test set and extracting the predicted labels:
predictions_test = trainer.predict(test_dataset)
pred_labels_test = np.argmax(predictions_test.predictions, axis=-1)

100%|██████████| 175/175 [00:34<00:00,  5.06it/s]


**Print Classification Reports For Evaluation:**

In [ ]:
print("Classification Report Train set: \n", classification_report(y_train, pred_labels_train))
print("Confusion Matrix Train set: \n", confusion_matrix(y_train, pred_labels_train))

Classification Report Train set: 
               precision    recall  f1-score   support

           0       0.83      0.97      0.89      2600
           1       0.81      0.40      0.54       867

    accuracy                           0.83      3467
   macro avg       0.82      0.69      0.72      3467
weighted avg       0.83      0.83      0.80      3467

Confusion Matrix Train set: 
 [[2521   79]
 [ 519  348]]


In [ ]:
print("Classification Report Test set: \n", classification_report(y_test, pred_labels_test))
print("Confusion Matrix Test set: \n", confusion_matrix(y_test, pred_labels_test))

Classification Report Test set: 
               precision    recall  f1-score   support

           0       0.89      0.91      0.90      1200
           1       0.40      0.35      0.37       200

    accuracy                           0.83      1400
   macro avg       0.65      0.63      0.64      1400
weighted avg       0.82      0.83      0.83      1400

Confusion Matrix Test set: 
 [[1096  104]
 [ 130   70]]


### **Testing With Try 3 Training Epochs:**

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3, # Experimenting with 3 epochs to see effect on results.
    per_device_train_batch_size=best_params['per_device_train_batch_size'],
    learning_rate=best_params['learning_rate'],
    weight_decay=best_params['weight_decay'],
    logging_dir='./logs',
    logging_steps=50,
    evaluation_strategy='no'
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

In [ ]:
# Training with the best hyperparameters:
trainer.train()

  4%|▍         | 50/1302 [00:49<18:09,  1.15it/s]

{'loss': 0.1604, 'grad_norm': 0.04119155928492546, 'learning_rate': 3.1242052282057315e-05, 'epoch': 0.12}


  8%|▊         | 100/1302 [01:36<23:58,  1.20s/it]

{'loss': 0.1193, 'grad_norm': 0.06762097775936127, 'learning_rate': 2.999436648804544e-05, 'epoch': 0.23}


 12%|█▏        | 150/1302 [02:41<18:39,  1.03it/s]

...

100%|█████████▉| 1300/1302 [24:51<00:01,  1.21it/s]

{'loss': 0.2763, 'grad_norm': 0.08760837465524673, 'learning_rate': 4.990743176047494e-08, 'epoch': 3.0}


100%|██████████| 1302/1302 [24:53<00:00,  1.15s/it]

{'train_runtime': 1493.132, 'train_samples_per_second': 6.966, 'train_steps_per_second': 0.872, 'train_loss': 0.1704139141573323, 'epoch': 3.0}


TrainOutput(global_step=1302, training_loss=0.1704139141573323, metrics={'train_runtime': 1493.132, 'train_samples_per_second': 6.966, 'train_steps_per_second': 0.872, 'total_flos': 801743580117000.0, 'train_loss': 0.1704139141573323, 'epoch': 3.0})

In [ ]:
# Predicting and extracting:
predictions_train = trainer.predict(train_dataset)
pred_labels_train = np.argmax(predictions_train.predictions, axis=-1)
predictions_test = trainer.predict(test_dataset)
pred_labels_test = np.argmax(predictions_test.predictions, axis=-1)

100%|██████████| 175/175 [00:35<00:00,  4.90it/s]


**Printing Evaluation Metrics For Comparing Performance With 3 Epochs:**

In [ ]:
print("Classification Report Train set: \n", classification_report(y_train, pred_labels_train))
print("Confusion Matrix Train set: \n", confusion_matrix(y_train, pred_labels_train))

Classification Report Train set: 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      2600
           1       0.99      0.95      0.97       867

    accuracy                           0.99      3467
   macro avg       0.99      0.97      0.98      3467
weighted avg       0.99      0.99      0.99      3467

Confusion Matrix Train set: 
 [[2593    7]
 [  43  824]]


In [ ]:
print("Classification Report Test set: \n", classification_report(y_test, pred_labels_test))
print("Confusion Matrix Test set: \n", confusion_matrix(y_test, pred_labels_test))

Classification Report Test set: 
               precision    recall  f1-score   support

           0       0.88      0.83      0.86      1200
           1       0.26      0.35      0.30       200

    accuracy                           0.76      1400
   macro avg       0.57      0.59      0.58      1400
weighted avg       0.79      0.76      0.78      1400

Confusion Matrix Test set: 
 [[996 204]
 [130  70]]


## **Step 7: Predict on some selected tweets**

In [ ]:
tweets = [
    ("Biden is a great President like none other we have had", 1),
    ("Max Verstappen is such a clean driver, he never makes dirty moves when racing", 1),
    ("I can't wait to spend time with my family over Christmas! I just love being the only single one and the many many questions asking when will I get a boyfriend 🙄", 1),
    ("Well this is awesome news to wake up to!", 0),
    ("So school have sent out a really helpful newsletter with all key dates and activities. So many fun things arranged for the children.", 0),
    ("I just love the smell of one million 😍", 0)
]

# Tokenize the tweets and create a dataset. Both ensuring that float32 is used
tweet_texts = [tweet for tweet, _ in tweets]
tweet_encodings = tokenizer(tweet_texts, padding=True, truncation=True, max_length=150, return_tensors='pt')
tweet_dataset = SarcasmDataset(tweet_encodings, labels=torch.zeros(len(tweets), dtype=torch.float32))

# Determine the device and move the model to the device
device = torch.device('mps' if torch.has_mps else 'cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Predict and print
predictions = trainer.predict(tweet_dataset)
pred_labels = np.argmax(predictions.predictions, axis=-1)

for tweet, label in zip(tweets, pred_labels):
    if label == 1:
        print(f"Tweet: {tweet}\nPrediction: Sarcastic\n")
    else:
        print(f"Tweet: {tweet}\nPrediction: Non-sarcastic\n")

Tweet: Biden is a great President like none other we have had
Prediction: Non-sarcastic

Tweet: Max Verstappen is such a clean driver, he never makes dirty moves when racing
Prediction: Non-sarcastic

Tweet: I can't wait to spend time with my family over Christmas! I just love being the only single one and the many many questions asking when will I get a boyfriend 🙄
Prediction: Non-sarcastic

Tweet: Well this is awesome news to wake up to!
Prediction: Non-sarcastic

Tweet: So school have sent out a really helpful newsletter with all key dates and activities. So many fun things arranged for the children.
Prediction: Non-sarcastic

Tweet: I just love the smell of one million 😍
Prediction: Non-sarcastic

